In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil

def parse(line):
    elems = line.split(' ')
    
    obj = {
        'timestamp': dateutil.parser.parse(elems[0])
    }
    for elem in elems[1:]: 
        (key,v) = elem.split(':')
        obj[key] = v
        
    return obj

def extract_byte(x, index):
    def byte_mapper(x):
        if not isinstance(x, basestring):
            return None
        if len(x) > 0:
            return ord(x.decode('hex')[index])
        else:
            return None
    return x.map(byte_mapper)

lines = open('all.txt', "rt").readlines()
column_order = ['timestamp', 'ID1', 'PTYPE', 'SEQ', 'ID2', 'CON', 'B9', 'BLEN', 'MTYPE', 'BODY', 'CRC']
packets = pd.DataFrame(map(lambda x: parse(x), lines), columns=column_order).set_index('timestamp')
packets.head()

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-17 20:23:30.966899,1f014829,POD,25,1f014829,NaN,28,10,1d18,02a82800002b7bff8134,45\n
2016-06-17 20:24:44.634776,1f014829,POD,28,1f014829,NaN,30,10,1d18,02a82800002b7fff0360,ea\n
2016-06-17 20:25:53.328967,1f014829,POD,31,1f014829,NaN,38,10,1d18,02a8a800002b87ff037c,f2\n
2016-06-17 20:48:44.551830,1f014829,POD,02,1f014829,NaN,00,10,1d18,02ad2800002bdfff009c,df\n
2016-06-17 20:48:47.828112,1f014829,POD,05,1f014829,NaN,08,10,1d18,02ad2800002bdfff83c5,cf\n


In [2]:
# Drop packets that have incorrect body length; likely corrupt packets
packets['blen_actual'] = packets['BODY'].map(lambda x: len(x) / 2, 'ignore').fillna(0).astype(np.int)
packets['BLEN'] = packets['BLEN'].map(lambda x: int(x), 'ignore').fillna(0)
ok_big_packets = (packets['blen_actual'] == 23) & (packets['BLEN'] > 23)
no_body_packets = (packets['BLEN'] == 0)
ok_small_packets = (packets['blen_actual'] == packets['BLEN']) & (packets['blen_actual'] > 0)
packets = packets.loc[ok_big_packets | no_body_packets | ok_small_packets]
packets = packets.drop('blen_actual', axis=1)

# Drop sequential duplicates
packets = packets[packets["SEQ"].shift(-1) != packets["SEQ"]]


In [3]:
packets.loc[packets["ID1"] == 'ffffffff']

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-30 20:52:58.930388,ffffffff,PDM,14,ffffffff,NaN,00,6.0,0704,1f0148288315,ec\n
2016-06-30 20:52:58.974498,ffffffff,POD,15,ffffffff,NaN,04,23.0,0115,02070002070002020000a5180005304e8d1f01482800c7,8c\n
2016-06-30 20:52:58.998169,ffffffff,ACK,16,1f014828,NaN,NaN,0.0,NaN,NaN,30\n
2016-06-30 20:52:59.787923,ffffffff,PDM,17,ffffffff,NaN,00,21.0,0313,1f0148281404061e1014340000a5180005304e82ea,03\n
2016-06-30 20:52:59.865898,ffffffff,POD,18,ffffffff,NaN,04,29.0,011b,13881008340a5002070002070002030000a5180005304e,59\n
2016-06-30 20:52:59.870056,ffffffff,ACK,19,1f014828,NaN,NaN,0.0,NaN,NaN,96\n
2016-06-30 20:52:59.997560,ffffffff,CON,20,NaN,1f0148280281,NaN,0.0,NaN,NaN,e7\n
2016-06-30 20:53:00.002081,ffffffff,ACK,21,1f014828,NaN,NaN,0.0,NaN,NaN,dd\n
2016-07-12 09:48:28.015762,ffffffff,PDM,0,ffffffff,NaN,00,6.0,0704,1f01482b831f,63\n


### Look for body crc
(body crc is different from the 'CRC' column (packet crc) in the dataframe)

We're focused on the last two bytes of the body, as they seem to change like a crc.

* byte -2: bit 7 and bits 1:0 toggle, but 6:2 are always 0
* byte -1: two quiet bits: 6 and 0

In [4]:
# Let's compare packets of the same length, first
packets['BLEN'].value_counts()

0.0      719
10.0     560
3.0      401
31.0      58
12.0      12
50.0       8
6.0        8
21.0       6
24.0       5
23.0       4
29.0       4
25.0       3
36.0       3
205.0      3
32.0       2
126.0      2
157.0      1
7.0        1
Name: BLEN, dtype: int64

In [5]:
packets["ID1"].copy().sort_values().unique()

array(['1f014828', '1f014829', '1f01482a', '1f01482b', '1f02d5ae',
       '1f02d5af', '1f039cec', 'effffff7', 'fffd7dfe', 'ffffffff'], dtype=object)

In [6]:
packets.loc[packets['MTYPE'] == '0704']

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-30 20:52:58.930388,ffffffff,PDM,14,ffffffff,NaN,00,6.0,0704,1f0148288315,ec\n
2016-07-12 09:48:28.015762,ffffffff,PDM,0,ffffffff,NaN,00,6.0,0704,1f01482b831f,63\n
2016-07-12 11:11:08.745305,ffffffff,PDM,0,ffffffff,NaN,00,6.0,0704,1f039cec0050,ca\n
2016-07-12 11:03:02.957165,ffffffff,PDM,14,ffffffff,NaN,00,6.0,0704,1f0148288315,ec\n
2016-07-12 11:03:09.284199,fffd7dfe,PDM,10,fffdffff,NaN,00,6.0,0704,1f0148280311,ec\n
2016-07-12 11:27:27.439881,ffffffff,PDM,8,ffffffff,NaN,00,6.0,0704,1f0148288315,5b\n


### xor each packet against all others, to identify bits that change (bit_deltas)

In [11]:
import itertools

# Compare packets
body_packets = packets.loc[packets['BLEN'] == 10]

# (re)assemble raw packet data

ptype_values = {
    'PDM': 0b101,
    'POD': 0b111,
    'ACK': 0b010,
    'CON': 0b100,
    }

blen_hex = body_packets["BLEN"].map(lambda x: chr(int(x)).encode('hex'))
ptype_val = body_packets["PTYPE"].map(lambda x: ptype_values[x] << 5)
seq_val = body_packets["SEQ"].map(lambda x: int(x))
b5_hex = (ptype_val + seq_val).map(lambda x: chr(x).encode('hex'))
crc_hex = body_packets["CRC"].map(lambda x: x.rstrip())
header_hex_data = body_packets["ID1"] + b5_hex
raw_hex_data = body_packets["ID2"] + body_packets["B9"] + blen_hex + body_packets["MTYPE"] + body_packets["BODY"]

print (header_hex_data + raw_hex_data + crc_hex).head()
raw_data = raw_hex_data.map(lambda x: x.decode('hex'))

def xor_strings(s1,s2):
    xored_bytes = [ord(a) ^ ord(b) for a,b in zip(s1,s2)]
    return [s1.encode('hex'),s2.encode('hex')] + list(np.unpackbits(np.array(xored_bytes, dtype=np.uint8)))

combinations = map(lambda x: xor_strings(x[0], x[1]), itertools.combinations(list(raw_data), 2))
bit_deltas = pd.DataFrame(combinations)

#
crc_split_point = -16
bit_cols = list(range(0,len(combinations[0])-2))
body_columns = map(lambda x: ('body', x), bit_cols[:crc_split_point])
crc_columns = map(lambda x: ('crc', x), bit_cols[crc_split_point:])
                
bit_deltas.columns = pd.MultiIndex.from_tuples([('data', 'p1'), ('data', 'p2')] + body_columns + crc_columns)
bit_deltas['num_body_deltas'] = bit_deltas['body'].sum(axis=1)
bit_deltas['num_crc_deltas'] = bit_deltas['crc'].sum(axis=1)

# Yes, this is unwieldy, but necessary to be able to scan columns visually
pd.options.display.max_columns = 200
bit_deltas

timestamp
2016-06-17 20:23:30.966899    1f014829f91f014829280a1d1802a82800002b7bff813445
2016-06-17 20:24:44.634776    1f014829fc1f014829300a1d1802a82800002b7fff0360ea
2016-06-17 20:25:53.328967    1f014829ff1f014829380a1d1802a8a800002b87ff037cf2
2016-06-17 20:48:44.551830    1f014829e21f014829000a1d1802ad2800002bdfff009cdf
2016-06-17 20:48:47.828112    1f014829e51f014829080a1d1802ad2800002bdfff83c5cf
dtype: object


data  \
                                          p1   
0       1f014829280a1d1802a82800002b7bff8134   
1       1f014829280a1d1802a82800002b7bff8134   
2       1f014829280a1d1802a82800002b7bff8134   
3       1f014829280a1d1802a82800002b7bff8134   
4       1f014829280a1d1802a82800002b7bff8134   
5       1f014829280a1d1802a82800002b7bff8134   
6       1f014829280a1d1802a82800002b7bff8134   
7       1f014829280a1d1802a82800002b7bff8134   
8       1f014829280a1d1802a82800002b7bff8134   
9       1f014829280a1d1802a82800002b7bff8134   
10      1f014829280a1d1802a82800002b7bff8134   
11      1f014829280a1d1802a82800002b7bff8134   
12      1f014829280a1d1802a82800002b7bff8134   
13      1f014829280a1d1802a82800002b7bff8134   
14      1f014829280a1d1802a82800002b7bff8134   
15      1f014829280a1d1802a82800002b7bff8134   
16      1f014829280a1d1802a82800002b7bff8134   
17      1f014829280a1d1802a82800002b7bff8134   
18      1f014829280a1d1802a82800002b7bff8134   
19      1f014829280a1d1802a82800002b7bff8134   
20      1f014829280a1d1802a82800002b7bff8134   
21      1f014829280a1d1802a82800002b7bff8134   
22      1f014829280a1d1802a82800002b7bff8134   
23      1f014829280a1d1802a82800002b7bff8134   
24      1f014829280a1d1802a82800002b7bff8134   
25      1f014829280a1d1802a82800002b7bff8134   
26      1f014829280a1d1802a82800002b7bff8134   
27      1f014829280a1d1802a82800002b7bff8134   
28      1f014829280a1d1802a82800002b7bff8134   
29      1f014829280a1d1802a82800002b7bff8134   
...                                      ...   
156490  1f01482b0c0a1d18002c50000000c3ff8151   
156491  1f01482b0c0a1d18002c50000000c3ff8151   
156492  1f01482b140a1d08002c20000000c7ff00a0   
156493  1f01482b140a1d08002c20000000c7ff00a0   
156494  1f01482b140a1d08002c20000000c7ff00a0   
156495  1f01482b140a1d08002c20000000c7ff00a0   
156496  1f01482b140a1d08002c20000000c7ff00a0   
156497  1f01482b140a1d08002c20000000c7ff00a0   
156498  1f01482b140a1d08002c20000000c7ff00a0   
156499  1f01482b1c0a1d0f002c30000000c7ff01c0   
156500  1f01482b1c0a1d0f002c30000000c7ff01c0   
156501  1f01482b1c0a1d0f002c30000000c7ff01c0   
156502  1f01482b1c0a1d0f002c30000000c7ff01c0   
156503  1f01482b1c0a1d0f002c30000000c7ff01c0   
156504  1f01482b1c0a1d0f002c30000000c7ff01c0   
156505  1f014828140a1d0300002000000003ff0319   
156506  1f014828140a1d0300002000000003ff0319   
156507  1f014828140a1d0300002000000003ff0319   
156508  1f014828140a1d0300002000000003ff0319   
156509  1f014828140a1d0300002000000003ff0319   
156510  1f0148281c0a1d4400003034000003ff8010   
156511  1f0148281c0a1d4400003034000003ff8010   
156512  1f0148281c0a1d4400003034000003ff8010   
156513  1f0148281c0a1d4400003034000003ff8010   
156514  1f014828240a1d050016c000000007ff80fe   
156515  1f014828240a1d050016c000000007ff80fe   
156516  1f014828240a1d050016c000000007ff80fe   
156517  1f0148282c0a1d160016d00000000bff8116   
156518  1f0148282c0a1d160016d00000000bff8116   
156519  1f014828340a1d160016e00000000bff8087   

                                             body                             \
                                          p2    0  1  2  3  4  5  6  7  8  9   
0       1f014829300a1d1802a82800002b7fff0360    0  0  0  0  0  0  0  0  0  0   
1       1f014829380a1d1802a8a800002b87ff037c    0  0  0  0  0  0  0  0  0  0   
2       1f014829000a1d1802ad2800002bdfff009c    0  0  0  0  0  0  0  0  0  0   
3       1f014829080a1d1802ad2800002bdfff83c5    0  0  0  0  0  0  0  0  0  0   
4       1f014829100a1d1802ad2800002bdfff014b    0  0  0  0  0  0  0  0  0  0   
5       1f0148291c0a1d1802ad2800002bdfff020a    0  0  0  0  0  0  0  0  0  0   
6       1f014829240a1d1802ada800002be7ff021c    0  0  0  0  0  0  0  0  0  0   
7       1f0148292c0a1d1802af2800002c07ff026c    0  0  0  0  0  0  0  0  0  0   
8       1f014829340a1d1802af2800002c07ff80e2    0  0  0  0  0  0  0  0  0  0   
9       1f0148293c0a1d1802af2800002c07ff03bb    0  0  0  0  0  0  0  0  0  0   
10      1f014829040a1d1802af2800002c07ff03e9    0  0  0

In [8]:
# This selects the number of 'collisions', or different frames with same crc.
# It could be used to identify bits not part of crc

(bit_deltas[(bit_deltas["num_body_deltas"] != 0 ) & (bit_deltas["num_crc_deltas"] == 0)]).head()


Empty DataFrame
Columns: [(data, p1), (data, p2), (body, 0), (body, 1), (body, 2), (body, 3), (body, 4), (body, 5), (body, 6), (body, 7), (body, 8), (body, 9), (body, 10), (body, 11), (body, 12), (body, 13), (body, 14), (body, 15), (body, 16), (body, 17), (body, 18), (body, 19), (body, 20), (body, 21), (body, 22), (body, 23), (body, 24), (body, 25), (body, 26), (body, 27), (body, 28), (body, 29), (body, 30), (body, 31), (body, 32), (body, 33), (body, 34), (body, 35), (body, 36), (body, 37), (body, 38), (body, 39), (body, 40), (body, 41), (body, 42), (body, 43), (body, 44), (body, 45), (body, 46), (body, 47), (body, 48), (body, 49), (body, 50), (body, 51), (body, 52), (body, 53), (body, 54), (body, 55), (body, 56), (body, 57), (body, 58), (body, 59), (body, 60), (body, 61), (body, 62), (body, 63), (body, 64), (body, 65), (body, 66), (body, 67), (body, 68), (body, 69), (body, 70), (body, 71), (crc, 72), (crc, 73), (crc, 74), (crc, 75), (crc, 76), (crc, 77), (crc, 78), (crc, 79), (crc, 80), (crc, 81), (crc, 82), (crc, 83), (crc, 84), (crc, 85), (crc, 86), (crc, 87), (num_body_deltas, ), (num_crc_deltas, )]
Index: []

In [9]:
# Try reshuffling crc bits, since we have bits 6:2 zeroed out in second to last byte
# Are those bits padding? Or?
def pack_crc_bits(hex_str):
    data = hex_str.decode('hex')
    pb = ord(data[-2])
    pb = (pb & 0b11) #+ (pb >> 5)
    data = data[:-2] + chr(pb) + data[-1]
    return data.encode('hex')
map(pack_crc_bits, raw_hex_data.head().values)

['1f01482a10030e0100002c',
 '1f01482a18030e01000322',
 '1f01482a20030e01000096',
 '1f01482a28030e01000398',
 '1f01482a30030e010003be']

In [13]:
# Look at low change rows
ones = bit_deltas[(bit_deltas["num_body_deltas"] == 1)]
map(lambda x: (x, x % 16, ones.loc[ones[('body',x)] == 1]["crc"].iloc[0:1].values), range(0,ones["body"].shape[1]))
#ones.loc[ones[('body',34)] == 1]
#ones

# twos
#twos = bit_deltas[(bit_deltas["num_body_deltas"] == 2)]
#twos.loc[(twos[('body',37)] == 1) & (twos[('body',36)] == 1)]
#twos

[(0, 0, array([], shape=(0, 16), dtype=int64)),
 (1, 1, array([], shape=(0, 16), dtype=int64)),
 (2, 2, array([], shape=(0, 16), dtype=int64)),
 (3, 3, array([], shape=(0, 16), dtype=int64)),
 (4, 4, array([], shape=(0, 16), dtype=int64)),
 (5, 5, array([], shape=(0, 16), dtype=int64)),
 (6, 6, array([], shape=(0, 16), dtype=int64)),
 (7, 7, array([], shape=(0, 16), dtype=int64)),
 (8, 8, array([], shape=(0, 16), dtype=int64)),
 (9, 9, array([], shape=(0, 16), dtype=int64)),
 (10, 10, array([], shape=(0, 16), dtype=int64)),
 (11, 11, array([], shape=(0, 16), dtype=int64)),
 (12, 12, array([], shape=(0, 16), dtype=int64)),
 (13, 13, array([], shape=(0, 16), dtype=int64)),
 (14, 14, array([], shape=(0, 16), dtype=int64)),
 (15, 15, array([], shape=(0, 16), dtype=int64)),
 (16, 0, array([], shape=(0, 16), dtype=int64)),
 (17, 1, array([], shape=(0, 16), dtype=int64)),
 (18, 2, array([], shape=(0, 16), dtype=int64)),
 (19, 3, array([], shape=(0, 16), dtype=int64)),
 (20, 4, array([], shape